
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"

tips_df = spark.read.csv(file_location, header=True, inferSchema=True)
tips_df

Out[1]: DataFrame[total_bill: double, tip: double, sex: string, smoker: string, day: string, time: string, size: int]

In [0]:
tips_df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
tips_df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
tips_df.columns

Out[4]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
### Handling Categorical Features
from pyspark.ml.feature import StringIndexer


In [0]:
# StringIndexer does the same as LabelEncoder in sklearn
indexer = StringIndexer(
    inputCols=['sex', 'smoker', 'day', 'time'],
    outputCols=['sex_indexed', 'smoker_indexed', 'day_indexed', 'time_indexed']
)
tips_df = indexer.fit(tips_df).transform(tips_df)
tips_df.show()


+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
tips_df.columns

Out[8]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed']

In [0]:
feature_assebler = VectorAssembler(inputCols=['tip','sex_indexed','smoker_indexed','day_indexed','time_indexed' ],
                outputCol='Independent Features')
output = feature_assebler.transform(tips_df)
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|Independent Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,1.0,0.0,1.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|(5,[0,3],[1.66,1.0])|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0| (5,[0,3],[3.5,1.0])|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|(5,[0,3],[3.31,1.0])|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,1.0,0.0,1.0...|
|     25.29|4.71|  Male|    No|S

In [0]:
final_data = output.select(['Independent Features', 'total_bill'])
final_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,1.0,0.0,1.0...|     16.99|
|(5,[0,3],[1.66,1.0])|     10.34|
| (5,[0,3],[3.5,1.0])|     21.01|
|(5,[0,3],[3.31,1.0])|     23.68|
|[3.61,1.0,0.0,1.0...|     24.59|
|(5,[0,3],[4.71,1.0])|     25.29|
| (5,[0,3],[2.0,1.0])|      8.77|
|(5,[0,3],[3.12,1.0])|     26.88|
|(5,[0,3],[1.96,1.0])|     15.04|
|(5,[0,3],[3.23,1.0])|     14.78|
|(5,[0,3],[1.71,1.0])|     10.27|
|[5.0,1.0,0.0,1.0,...|     35.26|
|(5,[0,3],[1.57,1.0])|     15.42|
| (5,[0,3],[3.0,1.0])|     18.43|
|[3.02,1.0,0.0,1.0...|     14.83|
|(5,[0,3],[3.92,1.0])|     21.58|
|[1.67,1.0,0.0,1.0...|     10.33|
|(5,[0,3],[3.71,1.0])|     16.29|
|[3.5,1.0,0.0,1.0,...|     16.97|
|      (5,[0],[3.35])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression

# train test split
train, test = final_data.randomSplit([0.90, 0.10])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='total_bill', maxIter=3000, 
                             elasticNetParam=0.99)
regressor = regressor.fit(train)

In [0]:
regressor.coefficients

Out[36]: DenseVector([4.1047, -0.7988, 1.7815, -0.3555, -1.7003])

In [0]:
regressor.intercept

Out[37]: 7.9971015822777884

In [0]:
pred = regressor.evaluate(test)

In [0]:
pred.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|      (5,[0],[1.75])|     17.82|15.180306718501122|
|       (5,[0],[3.0])|     20.45|20.311167530089218|
|      (5,[0],[3.39])|     11.61|21.911996103304702|
|(5,[0,2],[1.64,1.0])|     15.36| 16.51028954356995|
| (5,[0,2],[2.0,1.0])|     22.67|17.987977457307323|
|(5,[0,2],[10.0,1.0])|     50.81|50.825486651471124|
|(5,[0,3],[1.71,1.0])|     10.27|14.660634808390686|
| (5,[0,3],[2.5,1.0])|     12.54|17.903338841314362|
| (5,[0,3],[2.5,1.0])|     14.07|17.903338841314362|
|(5,[0,3],[2.55,1.0])|     23.95|18.108573273777886|
|(5,[0,3],[5.07,1.0])|     29.93|28.452388669939488|
|[1.0,1.0,1.0,0.0,...|      3.07|13.084519276711916|
|[1.0,1.0,1.0,3.0,...|      5.75|12.018066184293065|
|[1.1,1.0,1.0,0.0,...|      12.9|13.494988141638963|
|[1.67,1.0,0.0,1.0...|     10.33|13.697677731094934|
|[2.0,0.0,1.0,1.0,...|     14.48|17.6324930931

In [0]:
pred.r2

Out[40]: 0.6315050552107719